# A Codewars Kata

Presented at San Diego Python User Group on 2017-03-23

At codewars.com, you can practice your coding skills by solving small coding challenges called kata. After solving a kata, you can see the solutions of others who have solved the kata. Often you discover that there is a better and simpler solution to the problem, an by letting yourself be inspired by others you can improve your coding skills. Since solutions are accepted in many different languages, it's also a great way to learn new languages. 

This talk is about a kata that I enjoyed in particular. It's about generating the cards of the game "Spot It!" (also known a "Dobbel").

"Spot It!" is played with a deck of 55 cards. Each cards has eight figures and any pair of cards is such that the two cards have exactly one figure in common.

<img src="assets/spotitcards-1024x851.jpg" alt="spotit" width="300"/>

The problem is: How do you make a deck of cards? How do you distribute the figures (there are 57 in total) onto the cards in such a way that, given any two cards, they have exactly one figure in common?

At first, this looks like a typical discrete optimization problem: Create one card at the time, making sure that it has one figure in common with each of the cards that has already been made. If stuck, backtrack, until all 55 cards have been made. If that takes too long, try to come up with something clever!

However, a hint is provided: Think of a card as a point on a plane and the figures on the card as the lines that pass through that point. From geometry, we know that through any pair of distinct points, passes one and only one line. This translates to the cards as: Given any two cards, they have one and only one figure in common.

But, through one point pass an infinite number of lines, whereas on one card there are only eight figures. The analogy seems to break? Not if we apply a finite geometry!

## A little math background

First step in defining a finite geometry is to define a finite field, a.k.a. Galois field. For this problem will need the set of non-negative integers less than $p$, where $p$ is a prime. Let's call this set $GF(p)$. This set can be represented in python as `range(p)`. Addition is defined as addition modulo 7, i.e., the sum of two numbers $m$ and $n$ is $(m + n) \mod p$. Likewise for multiplication; the product of $m$ and $n$ is $m \cdot n \mod p$. For every number $m \in GF(p)$, there exists a number $n\in GF(p)$ such that $m + n \equiv 0 \mod p$, and for every number $m \in GF(p)$ other than zero, there exists a number $n\in GF(p)$ (the multiplicative inverse of $m$) such that $m \cdot n \equiv 1 \mod p$. For example, if $p = 7$, then 2 and 4 are inverse because $2 \cdot 4 \equiv 1 \mod 7$. Likewise, 3 and 5 are inverse, 1 is its own inverse and so is 6.

In the same way as we can define the Euclidean plane as the Cartesian product of $\mathbb{R} \times \mathbb{R}$, we can define a finite plane as $GF(p)\times GF(p)$.

The figure below is an illustration of $GF(7) \times GF(7)$. Each dot represents a point. The point $(0, 0)$ is the bottom-left point. The seven highlighted dots represent a line. An equation for this line is $4x + y + 2 = 0$. It is easy to verify that the highlighted points satisfy this equation. For example, the point $(2, 4)$ satisfies the equation because $4\cdot 2 + 4 + 2 \equiv 0 \mod 7$.

Every line has an equation of the form $ax + by + c = 0$, but the equation is not unique. The reason is that we can multiply all the coefficients by a same number and we get another equation for the same line. For example, the equations $x + 2y + 4 = 0$, $2x + 4y + 1= 0$, $3x + 6y + 5 = 0$, and $4x + y + 2 = 0$ are all equations of the same line. To guarantee a unique representation of each line by an equation $ax + by + c = 0$, we impose that $a = 1$ or $a = 0$, and, if $a = 0$, then $b = 1$. 

The figure shows that we can connect the points by continuous lines that wrap around the square (i.e., when a line reaches an edge, it continues at the opposite edge in the same direction). The slope of the yellow line is 3, which suggests that the seven points have slope 3. However, the magenta line has slope $-1/2$, which suggests that the seven points have slope $-1/2$. But in $GF(7)$, $-1/2 = -4 = 3$, so in $GF(7)$, the magenta line has slope 3 too. The cyan line has slope $2/3 = 2 \cdot 1/3 = 2\cdot 5 = 3$ in $GF(7)$. Again we get that the slope is 3. No matter how one connects the points, the slope is always represented by the number 3 in $GF(7)$.

<img alt="Finite geometry example" src="assets/finite_geometry.svg"/>

Let's describe this small piece of mathematics in code. First the set $GF(7)$:

In [2]:
p = 7
gf = range(p)

Then the set of points, the Cartesian product $GF(p)\times GF(p)$.

In [4]:
from itertools import product
points = set(product(gf, gf))

There are $p + 1$ different directions. We can define these directions as pairs as follows:

In [3]:
directions = {(1, b) for b in gf} | {(0, 1)}

Each line can be identified uniquely by an equation $ax + by + c = 0$, where $a, b$ defines the direction and $c$ is a value in $GF(p)$

In [4]:
lines = {(a, b, c) for a, b in directions for c in gf}

Let's number these lines.

In [5]:
line_nums = {line:i for i, line in enumerate(lines, start=1)}

For each point, we have a card, which is represented as the tuple of the line numbers of all the lines passing through the point.

In [6]:
cards_fin = {tuple(line_nums[(a, b, -(a*x + b*y) % p)] for a, b in directions) 
         for (x, y) in points}

When p = 7, we have 49 cards that satisfy our criteria, i.e., each card is an 8-tuple and any two cards have only one number in common (the number of the line that passes through the corresponding points). We need 6 more cards, at least. 

As for now, two lines with the same direction do not intersect. However, we can change that by introducing a "line at infinity" and, for each direction, a point on the line at infinity. Two lines with the same direction intersect at the point on the line at infinity that corresponds to their direction. For each point on the line at infinity, we an create a card of the line numbers of the lines that pass through it. 

The line at infinity cannot be represented by an equation, but we can still give it a number:

In [7]:
line_inf = len(lines) + 1

For each point on the line at infinity, we can create a card with the line numbers of all the lines that pass through it. 

In [8]:
cards_inf = {(*(line_nums[(a, b, c)] for c in gf), line_inf) 
             for a, b in directions}

Our final set of cards is simply the union of `cards_fin` and `cards_inf`:

In [9]:
cards = cards_fin | cards_inf

That's it, we're done! All we have done is to define `gf`, `points`, `directions`, `lines`, `line_nums`, `cards_fin`, `line_inf`, `cards_inf`, and finally `cards`, using one statement for each. It might seems as we have "defined our way out of the problem". Yet, we have a solution.

## Verification

Do we have at least 55 cards?

In [10]:
len(cards)

57

What is the total number of different "figures" on the cards?

In [11]:
len({fig for card in cards for fig in card})

57

Do all cards contain 8 figures?

In [12]:
all(len(card) == p + 1 for card in cards) 

True

And are the figures distinct?

In [13]:
all(len(set(card)) == len(card) for card in cards)

True

Do all pair of cards have exactly one figure in common?

In [14]:
from itertools import combinations

all(len(set(card1) & set(card2)) == 1 for card1, card2 in combinations(cards, 2))

True